In [269]:
from lib import read_file

from dataclasses import dataclass

from functools import cached_property


In [273]:
@dataclass
class Range:
    start: int
    length: int

    @cached_property
    def end(self) -> int:
        return self.start + self.length

@dataclass
class RangeMap:
    dest_start: int
    source_start: int
    length: int

    @cached_property
    def source_end(self) -> int:
        return self.source_start + self.length
    
    @cached_property
    def offset(self) -> int:
        return self.dest_start - self.source_start

    def __contains__(self, value: int | Range) -> bool:
        if isinstance(value, int):
            return self.source_start <= value < self.source_end
        else:
            return (not (value.end <= self.source_start or value.start >= self.source_end))
    
    def map_value(self, value: int) -> int:
        return self.dest_start + (value - self.source_start)
    
    def map_range(self, range: Range) -> list[Range]:
        """
        Map a range. In general this can result in up to three
        new ranges, on below the source range, one within the
        source range, and one above the source range.
        """
        ranges = []
        if range.start < self.source_start:
            # Have a range below the source range
            start = range.start
            end = min(range.end, self.source_start)
            ranges.append(Range(start, end - start))
        if range.end > self.source_end:
            # Have a range above the source range
            start = max(range.start, self.source_end)
            end = range.end
            ranges.append(Range(start, end - start))
        if (range.end >= self.source_start and range.start < self.source_end):
            # Have a range within the source range
            start = max(range.start, self.source_start) + self.offset
            end = min(range.end, self.source_end) + self.offset
            ranges.append(Range(start, end - start))

        return ranges

@dataclass
class Map:
    """
    Stores a number of range maps between a given source and destination.
    """
    source: str
    dest: str

    ranges: list[RangeMap]

    def map(self, value: int) -> int:
        for range in self.ranges:
            if value in range:
                return range.map_value(value)
            
        return value


In [276]:
Range(10, 1) in RangeMap(0, 0, 10)


False

In [277]:
lines = read_file("5_input.txt")

# Get seeds
seeds = list(map(int, lines[0].split(":")[1].strip().split()))
lines = lines[2:][::-1]
maps: dict[str, Map] = {}

# Get maps
while lines:
    line = lines.pop()
    if 'map' in line:
        source = line.split('-')[0]
        dest = line.split('-')[2].split()[0]
        m = Map(source, dest, [])
    elif line != "":
        m.ranges.append(RangeMap(*list(map(int, line.split()))))
    elif line == "":
        maps[source] = m

maps[source] = m


In [278]:
def get_loc(value: int) -> int:
    m = maps["seed"]
    dest = m.dest

    while True:
        value = m.map(value)
        if dest not in maps:
            break
        m = maps[dest]
        dest = m.dest

    return value


In [279]:
min([get_loc(seed) for seed in seeds])


173706076

# Part 2